In [1]:
import mne
import torch
import pickle
import os, shutil
import numpy as np
from tqdm import tqdm

from utils import load_mmidb_data_dict, get_fsaverage, get_labels, get_annotations, get_window_dict

In [2]:
def pick_and_rename_MMIDB_channels(raw):
    raw = raw.copy()
    mne.channels.rename_channels(raw.info, {'Fp1': 'FP1', 'Fp2': 'FP2', 'Fz': 'FZ', 'Cz': 'CZ', 'Pz': 'PZ'})

    
    if 'P7' in raw.ch_names:
        raw.rename_channels({'P7': 'T5'})
    if 'P8' in raw.ch_names:
        raw.rename_channels({'P8': 'T6'})

    EEG_20_div = [
                'FP1', 'FP2',
        'F7', 'F3', 'FZ', 'F4', 'F8',
        'T7', 'C3', 'CZ', 'C4', 'T8',
        'T5', 'P3', 'PZ', 'P4', 'T6',
                 'O1', 'O2'
    ]
    
    raw.pick_channels(ch_names=EEG_20_div)
    raw.reorder_channels(EEG_20_div)

    return raw


def get_raw(FILE, high_pass=0.5, low_pass=70, notch=60, SAMPLE_FREQ=256):
    raw = mne.io.read_raw_edf(FILE, verbose=False, preload=True)
    mne.datasets.eegbci.standardize(raw)  # Set channel names

    # set reference to average + projection
    raw = raw.set_eeg_reference(ref_channels='average', projection=True, verbose=False)

    # set montage
    montage = mne.channels.make_standard_montage('standard_1020')
    raw = raw.set_montage(montage)

    # resample
    raw.resample(SAMPLE_FREQ, npad="auto", verbose=False)

    # filter
    raw.filter(high_pass, low_pass, fir_design='firwin', verbose=False)
    raw.notch_filter(notch, fir_design='firwin', verbose=False)

    # pick and rename channels
    raw = pick_and_rename_MMIDB_channels(raw)

    return raw


def make_concepts_folders(DATA_PATH_CONCEPTS, bands_to_use, label_names):

    # remove and remake concepts folder
    print('Removing and remaking concepts folder: ', DATA_PATH_CONCEPTS)

    if os.path.exists(DATA_PATH_CONCEPTS):
        shutil.rmtree(DATA_PATH_CONCEPTS)
    os.mkdir(DATA_PATH_CONCEPTS)

    for hemisphere in ['lh', 'rh']:
        for band in bands_to_use:
        # make directory if it doesn't exist
            if not os.path.exists(f'{DATA_PATH_CONCEPTS}{band}_{hemisphere}'):
                os.makedirs(f'{DATA_PATH_CONCEPTS}{band}_{hemisphere}')

def normalize(x, minimum=-0.00125, maximum=0.00125):
    return (x - minimum) / (maximum - minimum) * 2 - 1

def save_concept(DATA_PATH_CONCEPTS, raw, concept, patient, run, idx, NUMBER_CHANNELS=20, NUMBER_SAMPLES=1024):

    x = torch.zeros((1, NUMBER_CHANNELS, NUMBER_SAMPLES))
    x[:, :NUMBER_CHANNELS-1, :] = torch.from_numpy(raw.copy().get_data()[:, :NUMBER_SAMPLES].reshape(1, NUMBER_CHANNELS-1, NUMBER_SAMPLES))
    x = normalize(x)
    x[:,NUMBER_CHANNELS-1,:] = torch.ones((1, NUMBER_SAMPLES)) * -1  
    
    picklePath = DATA_PATH_CONCEPTS + concept + '/' + patient + run + '_' + idx + '_' + concept + '.pkl'
    with open(picklePath, 'wb') as handle:
        pickle.dump(x, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [3]:
# argparse?
SNR = 100.0
PARCELLATION = 'HCPMMP1_combined'

# argparse
DATA_PATH_RAW = '/home/williamtheodor/Documents/DL for EEG Classification/data/eegmmidb (raw)/files/'
DATA_PATH_CONCEPTS = '/home/williamtheodor/Documents/DL for EEG Classification/data/left right concepts MMIDB/'
ACTIVTY_DICT_PATH = '/home/williamtheodor/Documents/DL for EEG Classification/data/'


activity_dict = load_mmidb_data_dict(ACTIVTY_DICT_PATH, PARCELLATION, SNR, chop=False)

patients_to_exclude = ['S088', 'S089', 'S090', 'S092', 'S104', 'S106']
patients = [key for key in activity_dict['Alpha'].keys() if key not in patients_to_exclude]

bands = list(activity_dict.keys())
bands_to_use = bands
#bands_to_use = ['Alpha', 'Gamma'] #argparse

baseline_runs = ['R01', 'R02']
task_runs = ['R03', 'R04', 'R07', 'R08', 'R11', 'R12']

runs_to_use = task_runs #argparse
resting_state = False # true if using baseline runs

codes = ['T0', 'T1', 'T2']
codes_to_use = ['T1', 'T2'] #argparse

subjects_dir, subject, trans, src_path, bem_path = get_fsaverage()
labels = get_labels(subjects_dir, parcellation_name=PARCELLATION)
label_names = [label.name for label in np.array(labels).flatten()]

make_concepts_folders(DATA_PATH_CONCEPTS, bands_to_use, label_names)

Removing and remaking concepts folder:  /home/williamtheodor/Documents/DL for EEG Classification/data/left right concepts MMIDB/


In [4]:
number_labels = len(label_names)
number_bands = len(bands_to_use)

hemispheres = ['lh', 'rh']

for patient in tqdm(patients):

    if not resting_state:
        baseline_run = 'R01'
        baseline_activity_mean = np.array([activity_dict[band][patient][patient+baseline_run]['T0'].mean(axis=0) for band in bands_to_use])
        baseline_activity_mean = baseline_activity_mean.reshape(number_bands, number_labels)

    for run in task_runs:

        EDF_FILE = DATA_PATH_RAW+f'{patient}/{patient}{run}.edf'
        raw = get_raw(EDF_FILE)

        annotations = get_annotations(EDF_FILE)

        annotation_dict = get_window_dict(raw, annotations)

        for code in ['T1', 'T2']:
            for raw_idx, raw in enumerate(annotation_dict[code]):

                power = np.array([activity_dict[band][patient][patient+run][code][raw_idx] for band in bands_to_use])

                power /= baseline_activity_mean

                power_left = power[:, :number_labels//2].mean(axis=1)
                power_right = power[:, number_labels//2:].mean(axis=1)

                power = np.array((power_left, power_right))

                concept_idx = np.unravel_index(power.argmax(), power.shape)

                most_active_band_idx = concept_idx[1]
                most_active_band = bands_to_use[most_active_band_idx]

                hemisphere_idx = concept_idx[0]
                hemisphere = hemispheres[hemisphere_idx]

                concept = most_active_band + '_' + hemisphere

                save_concept(DATA_PATH_CONCEPTS, raw, concept, patient, run, str(raw_idx), NUMBER_CHANNELS=20, NUMBER_SAMPLES=1024)

 92%|█████████▏| 95/103 [03:17<00:16,  2.08s/it]/tmp/ipykernel_9332/3476600872.py:26: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_edf(FILE, verbose=False, preload=True)
/tmp/ipykernel_9332/3476600872.py:26: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_edf(FILE, verbose=False, preload=True)
/tmp/ipykernel_9332/3476600872.py:26: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_edf(FILE, verbose=False, preload=True)
/tmp/ipykernel_9332/3476600872.py:26: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_edf(FILE, verbose=False, preload=True)
/tmp/ipykernel_9332/3476600872.py:26: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_edf(FILE, verbose=False, preload=True)
/tmp/ipykernel_9332/3476600

In [5]:
patient = 'S001'
run = 'R01'
code = 'T1'
raw_idx = 0

power_left = np.array([activity_dict[band][patient][patient+run][code][raw_idx][:23] for band in bands_to_use])
power_right = np.array([activity_dict[band][patient][patient+run][code][raw_idx][23:] for band in bands_to_use])

power_right.shape

KeyError: 'T1'

In [ ]:
activity_dict['Alpha']['S001']['S001R01'].keys()

In [ ]:
power_left[2].mean()